In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

# Set the partitions to 4 or 8. 
spark.conf.set("spark.sql.shuffle.partitions", 8)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/08 13:19:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/3/DelayedFlights.csv"
spark.sparkContext.addFile(url)
flights_df = spark.read.csv(SparkFiles.get("DelayedFlights.csv"), sep=",", header=True)

# Create a lookup table for the 500 cities. 
url_cities="https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/3/cities500.txt"
spark.sparkContext.addFile(url_cities)
df_lookup_geo = spark.read.csv(SparkFiles.get("cities500.txt"), sep="\t", header=True)

# Create a lookup table for the airport codes. 
url_airportCodes ="https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/3/airportCodes.csv"
spark.sparkContext.addFile(url_airportCodes)
df_lookup_codes = spark.read.csv(SparkFiles.get("airportCodes.csv"), sep=",", header=True)


In [4]:
# Look over the delayed flight data.
flights_df.show()

+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
| id|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|  0|2008|    1|         3|        4|   2003|      1955|   2211|      2225|       

In [5]:
# Look over the data of the 500 cities.
df_lookup_geo.show()

+---------+-------------------+-------------------+--------------------+--------+---------+-------------+------------+------------+----+-----------+-----------+-----------+-----------+----------+---------+----+--------------+-----------------+
|geonameid|               name|          asciiname|      alternatenames|latitude|longitude|feature_class|feature_code|country_code| cc2|admin1_code|admin2_code|admin3_code|admin4_code|population|elevation| dem|      timezone|modification_date|
+---------+-------------------+-------------------+--------------------+--------+---------+-------------+------------+------------+----+-----------+-----------+-----------+-----------+----------+---------+----+--------------+-----------------+
|  3038999|             Soldeu|             Soldeu|                null|42.57688|  1.66769|            P|         PPL|          AD|null|         02|       null|       null|       null|       602|     null|1832|Europe/Andorra|       2017-11-06|
|  3039154|          El 

In [6]:
# Look over the airport codes.
df_lookup_codes.show()

+--------------+--------------------+-----------+
|          City|             country|airportCode|
+--------------+--------------------+-----------+
|       Aalborg|             Denmark|        AAL|
|      Aalesund|              Norway|        AES|
|        Aarhus|             Denmark|        AAR|
|Abbotsford, BC|              Canada|        YXX|
|Abbotsford, BC|              Canada|        YXX|
|      Aberdeen|            Scotland|        ABZ|
|  Aberdeen, SD|                 USA|        ABR|
|       Abidjan|         Ivory Coast|        ABJ|
|   Abilene, TX|                 USA|        ABI|
|     Abu Dhabi|United Arab Emirates|        AUH|
|         Abuja|             Nigeria|        ABV|
|      Acapulco|              Mexico|        ACA|
|         Accra|               Ghana|        ACC|
|         Adana|              Turkey|        ADA|
|   Addis Ababa|            Ethiopia|        ADD|
|Adelaide, S.A.|           Australia|        ADL|
|          Aden|               Yemen|        ADE|


In [7]:
# Filter the airport codes to only contain rows whose `country` equals `USA`
df_lookup_city_name=df_lookup_codes.filter("country='USA'")
df_lookup_city_name.show(5)

+------------+-------+-----------+
|        City|country|airportCode|
+------------+-------+-----------+
|Aberdeen, SD|    USA|        ABR|
| Abilene, TX|    USA|        ABI|
|   Akron, OH|    USA|        CAK|
| Alamosa, CO|    USA|        ALS|
|  Albany, GA|    USA|        ABY|
+------------+-------+-----------+
only showing top 5 rows



In [8]:
# Filter the latitude and longitude DataFrame to only contain the 'name','latitude','longitude','admin1_code' fields and rows whose `country_code` equals `US`
df_lookup_geo=df_lookup_geo.select('name','latitude','longitude','admin1_code').filter("country_code='US'")
df_lookup_geo.show(5)

+--------------+--------+---------+-----------+
|          name|latitude|longitude|admin1_code|
+--------------+--------+---------+-----------+
|   Bay Minette|30.88296|-87.77305|         AL|
|          Edna|28.97859|-96.64609|         TX|
|Bayou La Batre|30.40352|-88.24852|         AL|
|     Henderson|32.15322|-94.79938|         TX|
|       Natalia|29.18968|-98.86253|         TX|
+--------------+--------+---------+-----------+
only showing top 5 rows



In [9]:
# Create temporary views for each of our DataFrames
flights_df.createOrReplaceTempView('delayed')
df_lookup_city_name.createOrReplaceTempView('lookup_city')
df_lookup_geo.createOrReplaceTempView('lookup_geo')


In [10]:
# First, join the airport codes lookup table to the delayed flight DataFrame 
# and add the city of origin and destination like we did in the instructor demonstration.  

start_time = time.time()

spark.sql("""
select a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
a.Dest,
c.City as Dest_City,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+---------------+----+---------------+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|    Origin_City|Dest|      Dest_City|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+---------------+----+---------------+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------

In [11]:
# Add the `origin_latitude` and `origin_longitude` fields by joining the `lookup_geo` view 
# to the `lookup_city` view and the delayed flight DataFrame.
# Note:  The two lookup views do not have matching columns, so we must be mindful what names are used when joining both views together.

start_time = time.time()

spark.sql("""
select a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name and trim(split(b.City,',')[1])=geo.admin1_code
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+---------------+---------------+----------------+----+---------------+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|    Origin_City|Origin_latitude|Origin_longitude|Dest|      Dest_City|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+---------------+---------------+----------------+----+---------------+--------+------+-------+---

In [12]:
# Finally, add the `dest_latitude` and `dest_longitude` fields by joining the `lookup_geo` view again as another alias, `geo_dest`.

start_time = time.time()

spark.sql("""
select a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+---------------+---------------+----------------+----+---------------+-------------+--------------+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|    Origin_City|Origin_latitude|Origin_longitude|Dest|      Dest_City|Dest_latitude|Dest_longitude|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+---------------+---------------+-------

In [13]:
# Run the same query with a Broadcast hint for either table
 
start_time = time.time()

spark.sql("""
select /*+ BROADCAST(lookup_geo) */ 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
""").show()
print("--- %s seconds ---" % (time.time() - start_time))

23/02/08 13:19:41 WARN HintErrorLogger: Count not find relation 'lookup_geo' specified in hint 'BROADCAST(lookup_geo)'.
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+---------------+---------------+----------------+----+---------------+-------------+--------------+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|    Origin_City|Origin_latitude|Origin_longitude|Dest|      Dest_City|Dest_latitude|Dest_longitude|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---

In [14]:
# Run a SQL query using a CTE here that does some aggregations on the new data.  
# The purpose of this SQL is to add some processing time.
# Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select /*+ BROADCAST(lookup_geo) */ 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin_City, avg(CarrierDelay) avgCarrierDelay from allColumns group by 1
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

23/02/08 13:19:42 WARN HintErrorLogger: Count not find relation 'lookup_geo' specified in hint 'BROADCAST(lookup_geo)'.


+-----------------+------------------+
|      Origin_City|   avgCarrierDelay|
+-----------------+------------------+
|     Amarillo, TX|             29.75|
|    Allentown, PA| 45.73529411764706|
|    Asheville, NC| 37.00854700854701|
|  Bloomington, IL| 26.61576354679803|
|      Atlanta, GA| 26.44772910507928|
|       Bangor, ME| 38.35087719298246|
|      Augusta, GA|30.810526315789474|
|  Albuquerque, NM|14.058823529411764|
|       Austin, TX|24.712374581939798|
|  Baton Rouge, LA| 50.08943089430894|
|    Baltimore, MD| 13.77671451355662|
|       Albany, GA| 37.48543689320388|
|       Albany, NY|17.555900621118013|
|       Boston, MA|14.291417165668662|
|        Akron, OH| 20.93778801843318|
|Atlantic City, NJ|             106.3|
|   Birmingham, AL|17.817460317460316|
|     Appleton, WI|36.542857142857144|
|   Alexandria, LA| 37.52252252252252|
|    Anchorage, AK| 24.08955223880597|
+-----------------+------------------+
only showing top 20 rows

--- 2.2673869132995605 seconds ---


In [15]:
# Cache your largest temporary view
# Note: when we use SparkSQL to cache a table, the table is immediately cached (no lazy evaluation), when using Pyspark it will not be cached until an action is ran.
spark.sql("cache table delayed")

DataFrame[]

In [16]:
# Check that your table is cached 
spark.catalog.isCached("delayed")

True

In [17]:
# Run the same query again with the data cached. This should greatly improve the run time.  
# Keep in mind we are not working with particularly large data here so the improvements may not be dramatic.

start_time = time.time()

spark.sql("""
with allColumns
(select /*+ BROADCAST(lookup_geo) */ 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin_City, avg(ArrDelay) avgDelay from allColumns group by 1
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


23/02/08 13:19:49 WARN HintErrorLogger: Count not find relation 'lookup_geo' specified in hint 'BROADCAST(lookup_geo)'.
+-----------------+------------------+
|      Origin_City|          avgDelay|
+-----------------+------------------+
|     Amarillo, TX|              63.3|
|    Allentown, PA| 50.94957983193277|
|    Asheville, NC|58.392405063291136|
|  Bloomington, IL| 46.89930555555556|
|      Atlanta, GA| 37.90491635370434|
|       Bangor, ME| 50.27329192546584|
|      Augusta, GA| 55.65714285714286|
|  Albuquerque, NM| 32.54347826086956|
|       Austin, TX|       37.19140625|
|  Baton Rouge, LA|          65.50625|
|    Baltimore, MD|  39.0767004341534|
|       Albany, GA| 50.25352112676056|
|       Albany, NY| 39.14365671641791|
|       Boston, MA| 47.71041369472183|
|        Akron, OH|55.928196147110334|
|Atlantic City, NJ| 64.64285714285714|
|   Birmingham, AL| 43.17042606516291|
|     Appleton, WI| 42.99324324324324|
|   Alexandria, LA|50.947712418300654|
|    Anchorage, AK| 37

In [18]:
# Cache the lookup table.
spark.sql("cache table lookup_geo")

DataFrame[]

In [19]:
# Run the same query again with the data cached. This should greatly improve the run time.

start_time = time.time()

spark.sql("""
with allColumns
(select /*+ BROADCAST(lookup_geo) */ 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin_City, avg(ArrDelay) avgDelay from allColumns group by 1
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

23/02/08 13:19:51 WARN HintErrorLogger: Count not find relation 'lookup_geo' specified in hint 'BROADCAST(lookup_geo)'.
+-----------------+------------------+
|      Origin_City|          avgDelay|
+-----------------+------------------+
|     Amarillo, TX|              63.3|
|    Allentown, PA| 50.94957983193277|
|    Asheville, NC|58.392405063291136|
|  Bloomington, IL| 46.89930555555556|
|      Atlanta, GA| 37.90491635370434|
|       Bangor, ME| 50.27329192546584|
|      Augusta, GA| 55.65714285714286|
|  Albuquerque, NM| 32.54347826086956|
|       Austin, TX|       37.19140625|
|  Baton Rouge, LA|          65.50625|
|    Baltimore, MD|  39.0767004341534|
|       Albany, GA| 50.25352112676056|
|       Albany, NY| 39.14365671641791|
|       Boston, MA| 47.71041369472183|
|        Akron, OH|55.928196147110334|
|Atlantic City, NJ| 64.64285714285714|
|   Birmingham, AL| 43.17042606516291|
|     Appleton, WI| 42.99324324324324|
|   Alexandria, LA|50.947712418300654|
|    Anchorage, AK| 37

In [20]:
# Remember to uncache the table as soon as you are done.
spark.sql("uncache table delayed")
spark.sql("uncache table lookup_geo")

DataFrame[]

In [21]:
# Verify that the table is no longer cached
if spark.catalog.isCached("delayed") or spark.catalog.isCached("lookup_geo"):
  print("a table is till cached")
else:
  print("all clear")

all clear
